In [1]:
import threading
import queue
from bs4 import BeautifulSoup
import requests
import urllib.parse

In [7]:
import threading
import queue
import csv
from bs4 import BeautifulSoup
import requests

class TitleScraper:
    def __init__(self, url):
        self.url = url
    
    def get_title(self):
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 ",
                "Accept-Language": "en-US,en;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive",
                "Upgrade-Insecure-Requests": "1",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
            }
            response = requests.get(self.url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            print(f"Scraped {self.url}")
            return soup.title.string if soup.title else None
        except Exception as e:
            print(f"Error scraping {self.url}: {e}")
            return None

def worker(url_queue, result_queue):
    while not url_queue.empty():
        url = url_queue.get()
        scraper = TitleScraper(url)
        title = scraper.get_title()
        result_queue.put((url, title))
        url_queue.task_done()

def main(input_csv, output_csv):
    url_queue = queue.Queue()
    result_queue = queue.Queue()

    # 读取输入文件中的URL并加入队列
    with open(input_csv, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            url = row[0]
            url_queue.put(url)

    # 创建并启动工作线程
    for _ in range(1000):  
        thread = threading.Thread(target=worker, args=(url_queue, result_queue))
        thread.daemon = True
        thread.start()

    # 等待所有任务完成
    url_queue.join()

    # 写入输出文件
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        while not result_queue.empty():
            url, title = result_queue.get()
            writer.writerow([url, title])

if __name__ == "__main__":
    input_csv = '../../data/00-raw-data/test.csv'  # Using raw string
    output_csv = 'output.csv'  # 输出文件名
    
    main(input_csv, output_csv)


Error scraping http://1337x.to/torrent/294349/American-Idol-S11E04-Auditions-4-HDTV-XviD-FQM-ettv/: 403 Client Error: Forbidden for url: http://1337x.to/torrent/294349/American-Idol-S11E04-Auditions-4-HDTV-XviD-FQM-ettv/Error scraping http://1337x.to/torrent/1124395/Fast-and-Furious-7-2015-HD-TS-XVID-AC3-HQ-Hive-CM8/: 403 Client Error: Forbidden for url: http://1337x.to/torrent/1124395/Fast-and-Furious-7-2015-HD-TS-XVID-AC3-HQ-Hive-CM8/

Error scraping http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/: 403 Client Error: Forbidden for url: http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/
Error scraping http://1337x.to/torrent/1110018/Blackhat-2015-RUSSIAN-720p-WEB-DL-DD5-1-H264-RUFGT/: 403 Client Error: Forbidden for url: http://1337x.to/torrent/1110018/Blackhat-2015-RUSSIAN-720p-WEB-DL-DD5-1-H264-RUFGT/
Error scraping http://1337x.to/torrent/1160078/Avengers-age-of-Ultron-2015-HQ-CAM-H264-AC3-MURD3R/: 403 Client Error